In [8]:
# 라이브러리 로드
# =============================
import os
import time
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [9]:
# 경로 설정
# =============================
FILES = {
    "역전": r"C:\Users\user\Desktop\데이터분석\정제\역전할머니_정제완료.csv",
    "금별": r"C:\Users\user\Desktop\데이터분석\정제\금별맥주_정제완료.csv",
}

OUTPUT_DIR = r"C:\Users\user\Desktop\데이터분석\출력"
os.makedirs(OUTPUT_DIR, exist_ok=True)

OUTPUT_FILES = {
    "역전": os.path.join(OUTPUT_DIR, "역전_리뷰수.csv"),
    "금별": os.path.join(OUTPUT_DIR, "금별_리뷰수.csv"),
}

CHECKPOINT_FILES = {
    "역전": os.path.join(OUTPUT_DIR, "역전_리뷰수_checkpoint.csv"),
    "금별": os.path.join(OUTPUT_DIR, "금별_리뷰수_checkpoint.csv"),
}

In [10]:
#  CSV 안전 로드 함수
# =============================
def safe_read_csv(path):
    if os.path.exists(path):
        return pd.read_csv(path)
    else:
        print(f"[WARN] 파일 없음: {path}")
        return pd.DataFrame()

In [11]:
# Selenium 드라이버 생성
# =============================================
def create_driver(headless=False):
    options = Options()
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 10)
    return driver, wait

In [12]:
#  리뷰 수 크롤링
# =============================
def get_review_count(place_id: int, driver, wait) -> int:
    url = f"https://pcmap.place.naver.com/place/{place_id}/review/visitor"
    try:
        driver.get(url)
        
        # iframe 전환
        try:
            iframe = wait.until(EC.presence_of_element_located((By.ID, "entryIframe")))
            driver.switch_to.frame(iframe)
        except:
            pass

        # 리뷰 섹션 로딩 대기
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.place_section")))
        time.sleep(0.5)  # 안정화 대기

        review_count = 0
        sections = driver.find_elements(By.CSS_SELECTOR, "div.place_section")
        for sec in sections:
            try:
                title = sec.find_element(By.CSS_SELECTOR, "div.place_section_header_title").text
                if "리뷰" in title:
                    em = sec.find_element(By.CSS_SELECTOR, "em.place_section_count")
                    count_text = em.text
                    review_count = int(re.sub(r"[^\d]", "", count_text))
                    break
            except:
                continue

        driver.switch_to.default_content()
        return review_count

    except Exception as e:
        print(f"[ERROR] place_id={place_id} | {e}")
        driver.switch_to.default_content()
        return 0

In [20]:
# 브랜드별 크롤링 처리
# =============================
def process_brand(input_csv, output_csv, checkpoint_csv, headless=False):
    print(f"\n▶ 처리 시작: {input_csv}")
    df = safe_read_csv(input_csv)
    if df.empty or "place_id" not in df.columns or "지점명" not in df.columns:
        print("[ERROR] place_id 또는 지점명 컬럼이 존재하지 않음")
        return

    df = df.drop_duplicates(subset="place_id").reset_index(drop=True)
    df["리뷰수"] = 0

    driver, wait = create_driver(headless=headless)

    for idx, row in df.iterrows():
        place_id = row["place_id"]
        review_count = get_review_count(place_id, driver, wait)
        df.at[idx, "리뷰수"] = review_count
        print(f"[{idx+1}/{len(df)}] place_id={place_id} | 리뷰수={review_count}")

        # 5개마다 체크포인트 저장
        if (idx + 1) % 5 == 0:
            df.to_csv(checkpoint_csv, index=False, encoding="utf-8-sig")
            print(f"✔ 체크포인트 저장: {checkpoint_csv}")

        time.sleep(1.5)  # 과부하 방지

    # 최종 CSV에는 place_id, 지점명, 리뷰수만 저장
    df_final = df[["place_id", "지점명", "리뷰수"]]
    df_final.to_csv(output_csv, index=False, encoding="utf-8-sig")
    print(f"\n✅ 최종 저장 완료: {output_csv}")

    driver.quit()

    # 체크포인트 삭제
    if os.path.exists(checkpoint_csv):
        os.remove(checkpoint_csv)
        print(f"🗑 체크포인트 삭제: {checkpoint_csv}")

In [ ]:
# 브랜드별 실행
# =============================
if __name__ == "__main__":
    for brand, input_csv in FILES.items():
        process_brand(
            input_csv=input_csv,
            output_csv=OUTPUT_FILES[brand],
            checkpoint_csv=CHECKPOINT_FILES[brand],
            headless=False
        )

### ***리뷰수 = 0 재크롤링***
1. 기존 역전_리뷰수.csv, 금별_리뷰수.csv 로드
2. 리뷰수 == 0 인 행 필터링
3. 해당 place_id만 get_review_count 재호출
4. 리뷰수 업데이트
5. 같은 CSV 파일에 덮어쓰기 저장

### ***재크롤링(중복 크롤링 발생x)***
1. Selenium 함수 실행
2. 리뷰 수 크롤링 실행
3. 리뷰수 = 0 재크롤링 실행
4. 네이버 감지 걸렸을 경우 재크롤링 코드 다시 돌리기

In [13]:
# =============================
# 리뷰수 = 0 재크롤링 전용
# =============================

TARGET_FILES = {
    "역전": r"C:\Users\user\Desktop\데이터분석\수집 후 출력(CSV)\역전_리뷰수.csv",
    "금별": r"C:\Users\user\Desktop\데이터분석\수집 후 출력(CSV)\금별_리뷰수.csv",
}

driver, wait = create_driver(headless=False)

for brand, path in TARGET_FILES.items():
    print(f"\n▶ [{brand}] 파일 로드")
    df = pd.read_csv(path)

    if "place_id" not in df.columns or "리뷰수" not in df.columns:
        print("[ERROR] 컬럼 구조 이상")
        continue

    zero_idx = df[df["리뷰수"] == 0].index
    print(f"▶ 리뷰수 0 대상 개수: {len(zero_idx)}")

    for idx in zero_idx:
        place_id = df.at[idx, "place_id"]

        try:
            new_count = get_review_count(place_id, driver, wait)

            if new_count > 0:
                df.at[idx, "리뷰수"] = new_count
                print(f"✔ place_id={place_id} | {new_count}")
            else:
                print(f"✖ place_id={place_id} | 여전히 0")

        except Exception as e:
            print(f"[ERROR] place_id={place_id} | {e}")

        time.sleep(1.5)

    # 🔥 같은 파일에 덮어쓰기
    df.to_csv(path, index=False, encoding="utf-8-sig")
    print(f"✅ [{brand}] 덮어쓰기 완료")

driver.quit()
print("\n🎯 전체 재크롤링 종료")


▶ [역전] 파일 로드
▶ 리뷰수 0 대상 개수: 1
✖ place_id=2036202855 | 여전히 0
✅ [역전] 덮어쓰기 완료

▶ [금별] 파일 로드
▶ 리뷰수 0 대상 개수: 5
✔ place_id=1284647505 | 80
✔ place_id=1875835175 | 189
✔ place_id=1418180719 | 1100
✔ place_id=1473371787 | 245
✔ place_id=1175386565 | 268
✅ [금별] 덮어쓰기 완료

🎯 전체 재크롤링 종료
